# Job Board Scraping Lab

In this lab you will first see a minimal but fully functional code snippet to scrape the LinkedIn Job Search webpage. You will then work on top of the example code and complete several chanllenges.

### Some Resources 

- [Requests library](http://docs.python-requests.org/en/master/#the-user-guide) documentation 
- [Beautiful Soup Doc](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Urllib](https://docs.python.org/3/library/urllib.html#module-urllib)
- [re lib](https://docs.python.org/3/library/re.html)
- [Scrapy](https://scrapy.org/)
- [List of HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)
- [HTML basics](http://www.simplehtmlguide.com/cheatsheet.php)
- [CSS basics](https://www.cssbasics.com/#page_start)

In [78]:
# Import the required libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

"""
This function searches job posts from LinkedIn and converts the results into a dataframe.
"""
def scrape_linkedin_job_search(keywords):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    # Assemble the full url with parameters
    scrape_url = ''.join([BASE_URL, 'keywords=', keywords])

    # Create a request to get the data from the server 
    page = requests.get(scrape_url)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    for card in soup.select("div.result-card__contents"):
        title = card.findChild("h3", recursive=False)
        company = card.findChild("h4", recursive=False)
        location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
        titles.append(title.string)
        companies.append(company.string)
        locations.append(location.string)
    
    # Inject job titles, companies, and locations into the empty dataframe
    zipped = zip(titles, companies, locations)
    for z in list(zipped):
        data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    # Return dataframe
    return data

In [79]:
# Example to call the function

results = scrape_linkedin_job_search('data%20analysis')
results

,Title,Company,Location
0,"Data Analytics Associate, CrossInstall",General Mills,"San Francisco, CA"
1,Data Scientist (Remote),Kraken Digital Asset Exchange,"Chicago, IL"
2,DATA SCIENTIST I,The Home Depot,"Houston, TX"
3,Junior Data Scientist Apprenticeship,IBM,"New York, NY"
4,Data Science - Intern,"Sonde Health, Inc.","Boston, MA"
5,Data Science Intern,Curology,"San Francisco, CA"
6,Data Analyst - Chicago,SpiderRock,"Chicago, IL"
7,Data Scientist 1,PayPal,"San Jose, CA"
8,Financial Analytics Consultant,Toyota North America,"Dallas, TX"
9,Research Assistant IV Non-Lab (Research Data A...,Harvard University,"Boston, MA"


## Challenge 1

The first challenge for you is to update the `scrape_linkedin_job_search` function by adding a new parameter called `num_pages`. This will allow you to search more than 25 jobs with this function. Suggested steps:

1. Go to https://www.linkedin.com/jobs/search/?keywords=data%20analysis in your browser.
1. Scroll down the left panel and click the page 2 link. Look at how the URL changes and identify the page offset parameter.
1. Add `num_pages` as a new param to the `scrape_linkedin_job_search` function. Update the function code so that it uses a "for" loop to retrieve several pages of search results.
1. Test your new function by scraping 5 pages of the search results.

Hint: Prepare for the case where there are less than 5 pages of search results. Your function should be robust enough to **not** trigger errors. Simply skip making additional searches and return all results if the search already reaches the end.

In [92]:
# your code here
def scrape_linkedin_job_search(keywords, num_pages): 
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    # Define new parameter
    titles = []
    companies = []
    locations = []
    
    # Assemble the full url with parameters (UPDATED)
    for page in range(num_pages):
        if page != 0:
            page *= 25
        scrape_url = ''.join([BASE_URL, 'keywords=', keywords, '&start=', str(page)])
        page = requests.get(scrape_url)
        soup = BeautifulSoup(page.text, 'html.parser')

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
        for card in soup.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)
        data = pd.DataFrame({"title": titles,
                            "company": companies,
                             "location": locations
        })

    # Return dataframe
    return data

In [93]:
results = scrape_linkedin_job_search('data%20analysis', 3)
results

,title,company,location
0,Business Analyst Level-1,Staffigo,"Cambridge, MA"
1,Business Analyst Level-1,Staffigo,"Bowie, MD"
2,Business Analyst Fresher,Staffigo,"Framingham, MA"
3,Urgent - Junior Level Business Analyst,Staffigo,"Boston, MA"
4,Process and Task mining Analyst,Zealogics LLC,"New York, NY"
...,...,...,...
70,DATA ANALYST INTERN,CARE,"Atlanta, GA"
71,Data Analyst,Instapage,"San Francisco, CA"
72,Data Analyst - Remote Position! - Dallas,Seasoned Recruitment,"Dallas, TX"
73,Quantitative Trading Analyst Intern,DRW,"Chicago, IL"


## Challenge 2

Further improve your function so that it can search jobs in a specific country. Add the 3rd param to your function called `country`. The steps are identical to those in Challange 1.

In [98]:
# your code here
def scrape_linkedin_job_search(keywords, num_pages, country): 
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    # Define new parameter
    titles = []
    companies = []
    locations = []
    
    # Assemble the full url with parameters (UPDATED)
    for page in range(num_pages):
        if page != 0:
            page *= 25
        scrape_url = ''.join([BASE_URL, 'keywords=', keywords, '&location=', country, '&start=', str(page)])
        page = requests.get(scrape_url)
        soup = BeautifulSoup(page.text, 'html.parser')

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
        for card in soup.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)
        data = pd.DataFrame({"title": titles,
                            "company": companies,
                             "location": locations
        })

    # Return dataframe
    return data

In [101]:
results = scrape_linkedin_job_search('data%20analysis', 3, 'Germany')
results

,title,company,location
0,Masterarbeit Statistics & Data Analysis,Boehringer Ingelheim,"Biberach, Baden-Württemberg, Germany"
1,"Analyst, Data Management and Analysis (Remote)",Tent Partnership for Refugees,"Frankfurt am Main, Hesse, Germany"
2,Data Analyst,Human Capital Advisory Group,Stuttgart Region
3,(Junior) Data Analyst *,myToys.de G.m.b.H,"Berlin, Berlin, Germany"
4,Data and Insights Analyst,TheSoul Publishing,Germany
...,...,...,...
70,Data Analyst / Data Engineer (m/w/d),uptodate Ventures GmbH,"Munich, Bavaria, Germany"
71,Data Analyst / Vertriebscontrolling / Data Sci...,Campusjäger,"Stuttgart, Baden-Württemberg, Germany"
72,Junior Data Analyst (m/w/d),Computer Futures,"Hamburg, Hamburg, Germany"
73,Data Analyst (f/m/x),audibene,"Berlin, Berlin, Germany"


## Challenge 3

Add the 4th param called `num_days` to your function to allow it to search jobs posted in the past X days. Note that in the LinkedIn job search the searched timespan is specified with the following param:

```
f_TPR=r259200
```

The number part in the param value is the number of seconds. 259,200 seconds equal to 3 days. You need to convert `num_days` to number of seconds and supply that info to LinkedIn job search.

In [104]:
# your code here
# your code here
def scrape_linkedin_job_search(keywords, num_pages, country, num_days): 
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    # Define new parameter
    titles = []
    companies = []
    locations = []
    seconds = 86400 * num_days
    
    # Assemble the full url with parameters (UPDATED)
    for page in range(num_pages):
        if page != 0:
            page *= 25
        scrape_url = ''.join([BASE_URL, 'f_TPR=r', str(seconds), 'keywords=', keywords, '&location=', country, '&start=', str(page)])
        page = requests.get(scrape_url)
        soup = BeautifulSoup(page.text, 'html.parser')
   
    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
        for card in soup.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)
        data = pd.DataFrame({"title": titles,
                            "company": companies,
                             "location": locations
        })

    # Return dataframe
    return data

In [105]:
results = scrape_linkedin_job_search('data%20analysis', 3, 'Germany', 1)
results

,title,company,location
0,Remote Data Entry Clerk - Work at Home,WW-Recruit,"Hamburg, Germany"
1,Modeling Engineer (d/f/m),Henkel,"Düsseldorf, North Rhine-Westphalia, Germany"
2,Account Executive,IG Recruit,"Munich, Bavaria, Germany"
3,Managing Director,ACL Partners,"Frankfurt, Hesse, Germany"
4,Director Business Innovation Consulting (all g...,Zühlke Group,Stuttgart Region
...,...,...,...
70,Digital Merchandising Assistant,SharkNinja,"Frankfurt am Main, Hesse, Germany"
71,Sales And Marketing Specialist,Do-Techs UG,"Constance, Baden-Württemberg, Germany"
72,Financial Controller,expand executive search,"Munich, Bavaria, Germany"
73,Head Of Marketing,AZUM system,Germany


## Bonus Challenge

Allow your function to also retrieve the "Seniority Level" of each job searched. Note that the Seniority Level info is not in the initial search results. You need to make a separate search request for each job card based on the `currentJobId` value which you can extract from the job card HTML.

After you obtain the Seniority Level info, update the function and add it to a new column of the returned dataframe.

In [ ]:
# your code here